***
# Projet n°7 - Implémentez un modèle de scoring
***
Notebook Jupyter réalisé par Dominique LEVRAY en Juillet/Août 2024

**3ème Notebook - Test_API**

Ce notebook intègre :
- Les tests de l'API de prédiction du modèle de scoring
***


In [18]:
# Importation des modules dont on aura besoin
import os
import time
import re
import numpy as np
import pandas as pd
import matplotlib as pl
import matplotlib.pyplot as plt
import seaborn as sns
import pprint

# Change le format d'affichage des floats pour être plus lisible
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Force l'affichage de toutes les colonnes et de toutes les lignes dans les fonctions head/tail/dtypes/etc... 
pd.set_option('display.max_columns',  None)     # Afficher toutes les colonnes
pd.set_option('display.max_rows',     None)     # Afficher toutes les lignes
pd.set_option('display.max_colwidth', 40)       # Limite sur la largeur des colonnes

# Défini les tailles par défaut des polices de caractères pour tous les diagrammes matplotlib
plt.rcParams.update({'font.size': 10}) 
plt.rcParams.update({'axes.titlesize': 12})
plt.rcParams.update({'axes.labelsize': 10})
plt.rcParams.update({'xtick.labelsize': 8})
plt.rcParams.update({'ytick.labelsize': 8})
plt.rcParams.update({'legend.fontsize': 8})

# Défini les grilles par défaut pour tous les diagrammes matplotlib
plt.rcParams.update({"axes.grid" : True, 'grid.color': 'green', 'grid.linestyle':'--', 'grid.linewidth':0.5})

In [3]:
###############################################################################################################
# Définir quelques fonctions utilitaires
###############################################################################################################

def read_csv_file(filename):
    """ 
     Cree un dataframe depuis un fichier .csv
    """
    print(f"\nLecture du fichier csv : data/{filename}")
    data_df = pd.read_csv("data/"+filename,    # Nom du fichier
                          sep      = ',',         # Séparateur virgule
                          encoding = 'utf-8',     # Encodage UTF-8
                         )
    print(f"\tLe fichier {filename} contient {data_df.shape[0]} lignes et {data_df.shape[1]} colonnes.")
    return data_df

#####################################################

def write_csv_file(filename, data_df):
    """ 
     Cree un fichier .csv depuis un dataframe
    """
    print(f"Ecriture du fichier csv : data/{filename}\n")
    data_df.to_csv("data/"+filename,       # Nom du fichier
                   sep=',',                 # Séparateur virgule
                   encoding = 'utf-8',      # Encodage UTF-8
                   index=False,             # N'inclus pas l'index
                  )